
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Project - Motor Control 
    

<br/>


In [ ]:
!pip install --upgrade tdmclient

In [ ]:
!pip install tqdm scipy

In [ ]:
!pip install numpy matplotlib pandas scipy tqdm

<strong>Imports :

In [1]:
import os
import sys
import math
from statistics import mean
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [2]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [3]:
await tdmclient.notebook.stop()

In [4]:
@onevent
def angle_to_rotate(pos_i, pos_f, theta_i):
    a = pos_f-pos_i
    theta_f = np.arctan2(a[1], a[0])
    r_theta = (theta_f*180)/np.pi-theta_i
    return r_theta
    
pos_thymio = np.array([1,1])
pos_goal = np.array([30,45])
theta = angle_to_rotate(pos_thymio,pos_goal,0)
print(theta)

56.61148642388849


In [5]:
@onevent
def distance_to_cover(pos_i, pos_f):
    distance = np.linalg.norm(pos_f-pos_i)
    return distance

pos_thymio = np.array([0,0])
pos_goal = np.array([30,45])
theta = angle_to_rotate(pos_thymio,pos_goal,0)
dist = distance_to_cover(pos_thymio,pos_goal)
print(dist)

54.08326913195984


In [6]:
nb_node = 3
initial_orientation = 0 #degree
positions = np.array([[0,0],[30,45],[50,50],[17,15],[50,10]]);
rotations = np.zeros(positions.shape[0]-1)
distances =  np.zeros(positions.shape[0]-1)

rotations[0] = angle_to_rotate(positions[0],positions[1],initial_orientation)
orientation = initial_orientation+rotations[0]

for i in range(1,positions.shape[0]-1):
    rotations[i] = angle_to_rotate(positions[i],positions[i+1],orientation)
    distances[i-1] = distance_to_cover(positions[i-1],positions[i])
    orientation = orientation+rotations[i]

distances[distances.size-1]=distance_to_cover(positions[distances.size-1],positions[distances.size])

print('Rotations :',rotations)
print('Distances :',distances)

Rotations : [  56.30993247  -42.27368901 -147.35155915  124.6996675 ]
Distances : [54.08326913 20.61552813 48.10405388 33.37663854]


In [7]:
print(3537/121)

29.231404958677686


## Async implementation:

In [8]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 6af373fe-a6df-4b9c-a9aa-8cde416dd09c

In [9]:
await node.wait_for_variables()
node.var

{'_fwversion': [14, 0],
 '_id': [18327],
 '_imot': [0, 0],
 '_integrator': [0, 0],
 '_productId': [8],
 '_vbat': [681, 684],
 'acc': [0, 0, 21],
 'acc._tap': [0],
 'button.backward': [0],
 'button.center': [0],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [11171, 9583, 14550, 14490, 15862],
 'buttons._noise': [95, 82, 120, 112, 126],
 'buttons._raw': [11155, 9571, 14530, 14470, 15851],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [18327],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [265],
 'mic.intensity': [35],
 'mic.threshold': [0],
 'motor.left.pwm': [0],
 'motor.left.speed': [0],
 'motor.left.target': [0],
 'motor.right.pwm': [0],
 'motor.right.speed': [0],
 'motor.right.target': [0],
 'prox.comm.rx

In [10]:
def angle_to_rotate(pos_i, pos_f, theta_i):
    a = pos_f-pos_i
    theta_f = np.arctan2(a[1], a[0])
    r_theta = (theta_f*180)/np.pi-theta_i
    return r_theta

def distance_to_cover(pos_i, pos_f):
    distance = np.linalg.norm(pos_f-pos_i)
    return distance

In [11]:
def path(positions, initial_orientation):
    nb_node = 3
    rotations = np.zeros(positions.shape[0]-1)
    distances =  np.zeros(positions.shape[0]-1)
    
    rotations[0] = angle_to_rotate(positions[0],positions[1],initial_orientation)
    orientation = initial_orientation+rotations[0]
    
    for i in range(1,positions.shape[0]-1):
        rotations[i] = angle_to_rotate(positions[i],positions[i+1],orientation)
        distances[i-1] = distance_to_cover(positions[i-1],positions[i])
        orientation = orientation+rotations[i]

    distances[distances.size-1]=distance_to_cover(positions[distances.size-1],positions[distances.size])
    rotation_ratios = rotations/360
    return distances, rotations, orientation, rotation_ratios


<strong>Kalman filter : 

In [12]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [13]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm

<strong> Variance for position :

In [14]:
qpx = 0.04 # variance on position state
qpy = 0.04
rpx = 0.25 # variance on position measurement 
rpy = 0.25

qpx = 0.04 # variance on position state
qpy = 0.04 
rpx = 0.25 # variance on position measurement 
rpy = 0.25


<strong>Variance for angle : 

In [15]:
std_speed = 11.77
q_theta_nu = std_speed/2 # variance on speed state
r_theta_nu = std_speed/2 # variance on speed measurement 

print(q_theta_nu,r_theta_nu)

5.885 5.885


In [16]:
# Initialising the remaining constants
# units: length [mm], time [s]
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
Q = np.array([[qpx, 0, 0], [0, qpy, 0],[0, 0, q_theta_nu]]); #w matrix
rotation_conv_factor = 0.41; #0.8 for 50 speed, 0.41 for 100 speed
speed_conv_factor = 0.338;

speed_conv_factor = 0.374 #mean of the two
transition_thresh = 500
l = 100 #mm, distance between the two wheels

def kalman_filter( x_est_prev, P_est_prev, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0,):

    ## Prediciton through the a priori estimate
    # estimated mean of the state
    orientation = np.double(x_est_prev[2][0]%360)*2*np.pi/360
    #x_est_prev[2][0] = orientation
    B = np.array([[Ts*np.cos(orientation)/2,Ts*np.cos(orientation)/2],
         [Ts*np.sin(orientation)/2,Ts*np.sin(orientation)/2],
         [-Ts*360/(l*2*np.pi), Ts*360/(l*2*np.pi)]])
    U = np.array([[Vr*speed_conv_factor],[Vl*speed_conv_factor]])
    x_est_a_priori = np.dot(A, x_est_prev) + np.dot(B, U);
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T));
    P_est_a_priori = P_est_a_priori + Q if type(Q) != type(None) else P_est_a_priori
    
    ## Update         
    # y, C, and R for a posteriori estimate, depending on transition
    if (camera_x != 0) : #XOR (one or the other but not both)
        # transition detected aka camera provides the position
        y = np.array([[camera_x],[camera_y],[camera_angle]])
        H = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
        R = np.array([[rpx, 0, 0],[0, rpy,0],[0, 0, r_theta_nu]]) #not the value of the variance of the position measurments
        
        # innovation / measurement residual
        i = y - np.dot(H, x_est_a_priori); #measured-mean aka difference between the two states
        # measurement prediction covariance
        S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R;
                 
        # Kalman gain (tells how much the predictions should be corrected based on the measurements)
        K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)));
            
        # a posteriori estimate
        x_est = x_est_a_priori + np.dot(K,i);
        P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori));

    else:
        x_est = x_est_a_priori
        P_est = P_est_a_priori
     
    return x_est, P_est

Test of the filter

In [17]:
#test of the kalman filter
x_est = np.array([[0], [0], [0]]) # in mm !!!
#P_est = [1000 * np.ones(2)]

P_est = 0
Vr = 100;
Vl = 100;

Ts = 0.1

camera_x = 4
camera_y = 0
camera_angle = 0

#for i in range(0,20):
new_x_est, new_P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0);
x_est = new_x_est;
P_est = new_P_est;

print(x_est[2][0])
print("new_x_est\n",new_x_est,"\n new_P_est\n", new_P_est)

8.69733149685918e-17
new_x_est
 [[3.7400000e+00]
 [0.0000000e+00]
 [8.6973315e-17]] 
 new_P_est
 [[0.04  0.    0.   ]
 [0.    0.04  0.   ]
 [0.    0.    5.885]]


In [18]:
import time

In [ ]:
thymio_data = []
initial_orientation = 0 #degree
positions = np.array([[0,0],[200,150]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 5 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
read = 1


async def main():    

    global x_est, P_est, Ts, read
    TURN = 1
    STRAIGHT = 0
    STATE = 0
    await node.wait_for_variables() # wait for Thymio variables values
    rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
    
    try:
        for i in range(0,20):
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[1],x_est[2][0]);
            if (abs(angle)<angle_threshold):
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
    
            else:
                if angle>0:
                    right = 1
                else:
                    right = 0

                turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
                await turn_task
    finally:
        rt.stop()
        node.send_set_variables(motors(0, 0))

await main()

Main to test things :

In [50]:
thymio_data = []
initial_orientation = 0 #degree
positions = np.array([[0,0],[200,150]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 1 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
read = 1


async def main():    

    global x_est, P_est, Ts, read
    TURN = 1
    STRAIGHT = 0
    STATE = 0
    await node.wait_for_variables() # wait for Thymio variables values
    rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
    
    ONETURN_RIGHT = 8.47 #s/turn #8.599
    ONETURN_LEFT = 8.94 #s/turn
    
    try:
        for i in range(0,50):
            print(x_est)
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[1],x_est[2][0]);
            print("to turn", angle)
            print("orientation : ",x_est[2][0])
            angle = angle / 360
            print("angle: ", angle)
            if angle>0:
                right = 1
                angle = angle*ONETURN_RIGHT
            else:
                right = 0
                angle = angle*ONETURN_LEFT
            t_turn = min(Ts-0.03, angle)
            turn_task = asyncio.create_task(turn_for_seconds(client, t_turn, right))
            await turn_task
            t_straight = max(0, Ts-0.03-t_turn)
            drive_task = asyncio.create_task(drive_for_seconds(client, t_straight))
            await drive_task
            await client.sleep(0.03)
            '''if (abs(angle)<angle_threshold) and (STATE == TURN):
                print("aaaaaaaa", abs(angle))
                print(STATE)
                read = 0
                time.sleep(10)
                read = 1
                STATE = STRAIGHT
                print("I wait for straight")
            elif (abs(angle)<angle_threshold) and (STATE == STRAIGHT):
                #straight for Ts=0.1 seconds
                print("straight")
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
    
            else:
                if STATE == STRAIGHT:
                    read = 0
                    time.sleep(1)
                    read = 1
                    print("I wait for turn")
                    STATE = TURN
                else:
                    if angle>0:
                        right = 1
                        angle = angle*ONETURN_RIGHT
                    else:
                        right = 0
                        angle = angle*ONETURN_LEFT
    
                    turn_task = asyncio.create_task(turn_for_seconds(client, min(Ts, angle), right))
                    await turn_task'''
    finally:
        rt.stop()
        node.send_set_variables(motors(0, 0))

await main()

[[0]
 [0]
 [0]]
to turn 36.86989764584402
orientation :  0
angle:  0.10241638234956672
[[0]
 [0]
 [0]]
			 Vr, Vl:  0 0
[[0.]
 [0.]
 [0.]]
			 Vr, Vl:  0 0
[[0.]
 [0.]
 [0.]]
			 Vr, Vl:  -16 11
[[-0.0935    ]
 [ 0.        ]
 [ 0.57857278]]
			 Vr, Vl:  -71 80
[[7.47914193e-02]
 [1.69946452e-03]
 [3.81429463e+00]]
to turn 33.065578407671694
orientation :  3.814294633744916
angle:  0.09184882891019915
[[7.47914193e-02]
 [1.69946452e-03]
 [3.81429463e+00]]
			 Vr, Vl:  -71 80
[[0.24271862]
 [0.01289526]
 [7.05001649]]
			 Vr, Vl:  38 43
[[1.74596656]
 [0.19880309]
 [7.15715959]]
			 Vr, Vl:  38 43
[[3.24886422]
 [0.38752166]
 [7.2643027 ]]
			 Vr, Vl:  -65 52
[[3.00771549]
 [0.35678249]
 [9.77145142]]
			 Vr, Vl:  -65 52
[[ 2.76814224]
 [ 0.31552393]
 [12.27860014]]
to turn 24.917224588328885
orientation :  12.278600141212566
angle:  0.06921451274535802
[[ 2.76814224]
 [ 0.31552393]
 [12.27860014]]
			 Vr, Vl:  -6 22
[[ 3.06049806]
 [ 0.37915343]
 [12.87860154]]
			 Vr, Vl:  0 0
[[ 3.060

[[38.4080057 ]
 [24.68463   ]
 [43.30724413]]
			 Vr, Vl:  69 55
[[40.09536409]
 [26.27511895]
 [43.00724343]]
			 Vr, Vl:  69 55
[[41.79102711]
 [27.85675113]
 [42.70724273]]
			 Vr, Vl:  16 -5
[[41.94218141]
 [27.99626768]
 [42.25724167]]
			 Vr, Vl:  86 -83
[[41.98370288]
 [28.03399271]
 [38.63580463]]
			 Vr, Vl:  86 -83
[[42.0275243 ]
 [28.06901974]
 [35.01436759]]
			 Vr, Vl:  86 -83
[[42.07347066]
 [28.1012089 ]
 [31.39293055]][[42.07347066]
 [28.1012089 ]
 [31.39293055]]
to turn 6.270552034389482
orientation :  31.39293055301291
angle:  0.01741820009552634

			 Vr, Vl:  83 32
[[43.90916988]
 [29.22141363]
 [30.30007085]]
			 Vr, Vl:  83 32
[[45.76590067]
 [30.30640258]
 [29.20721116]]
			 Vr, Vl:  97 68
[[48.45911226]
 [31.81203253]
 [28.58578113]]
			 Vr, Vl:  -72 69
[[48.40985075]
 [31.78519014]
 [31.60721677]]
			 Vr, Vl:  -72 69
[[48.36207258]
 [31.75578852]
 [34.62865241]]
to turn 3.317737891516579
orientation :  34.62865240523472
angle:  0.009215938587546053
[[48.36207258

[[108.16206781]
 [ 78.61280744]
 [ 46.02867906]]
			 Vr, Vl:  76 83
[[110.22642472]
 [ 80.75265404]
 [ 46.17867941]]
			 Vr, Vl:  76 83
[[112.28517244]
 [ 82.89789778]
 [ 46.32867976]]
			 Vr, Vl:  89 -83
[[112.36264883]
 [ 82.97905349]
 [ 42.64295686]]
			 Vr, Vl:  89 -83
[[112.44518196]
 [ 83.05506087]
 [ 38.95723396]]
			 Vr, Vl:  89 -83
[[112.53243042]
 [ 83.12560552]
 [ 35.27151105]]
			 Vr, Vl:  89 -83
[[112.62403328]
 [ 83.1903956 ]
 [ 31.58578815]]
			 Vr, Vl:  89 -83
[[112.71961162]
 [ 83.24916312]
 [ 27.90006524]]
to turn 9.508195204771738
orientation :  27.900065242336346
angle:  0.02641165334658816
[[112.71961162]
 [ 83.24916312]
 [ 27.90006524]]
			 Vr, Vl:  93 74
[[115.47952164]
 [ 84.71046428]
 [ 27.49292143]]
			 Vr, Vl:  0 0
[[115.47952164]
 [ 84.71046428]
 [ 27.49292143]]
			 Vr, Vl:  0 0
[[115.47952164]
 [ 84.71046428]
 [ 27.49292143]]
			 Vr, Vl:  0 0
[[115.47952164]
 [ 84.71046428]
 [ 27.49292143]]
			 Vr, Vl:  -69 65
[[115.41316897]
 [ 84.67593368]
 [ 30.36435672]

In [19]:
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

In [20]:
import asyncio
import time
    
async def drive_for_seconds(client, seconds):
    # Set the motor speeds
    v = {
        "motor.left.target": [int(99)],
        "motor.right.target": [int(100)],
    }
    await node.set_variables(v)
    await asyncio.sleep(seconds)

    #to delete probably 
    vstop = {
        "motor.left.target": [int(0)],
        "motor.right.target": [int(0)],
    }
    await node.set_variables(vstop)


async def turn_for_seconds(client, seconds,right):
    # Set the motor speeds
    if seconds != 0:
        if right==1:
            v = {
                "motor.left.target": [int(100)],
                "motor.right.target": [int(-100)],
            }
        else:
            v = {
                "motor.left.target": [int(-100)],
                "motor.right.target": [int(100)],
            }
        await node.set_variables(v)
    await asyncio.sleep(abs(seconds))

    #to delete probably
    vstop = {
        "motor.left.target": [int(0)],
        "motor.right.target": [int(0)],
    }
    await node.set_variables(vstop)

In [48]:
def filter():#x_est, P_est, Ts):
    global x_est, P_est, Ts, read
    #thymio_data = []
    #print(x_est)
    #if read = 1:
    print(x_est) 
    thymio_data.append({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})
    Vl = thymio_data[-1]["left_speed"]
    Vr = thymio_data[-1]["right_speed"]
    #else: 
    #    Vr = 0
    #    Vl = 0

    print("\t\t\t Vr, Vl: ", Vr, Vl)
    
    x_est, P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0)
    #print(x_est)
    

In [22]:
node.send_set_variables(motors(0, 0))
